In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
import json
import re

DB_CONFIG = {
    'host': 'localhost',
    'database': 'SeuBancoDeDados',
    'user': 'SeuUsuario',
    'password': 'SuaSenha',
    'port': 5432
}

def conectar_db():
    """Estabelece conexão com o banco de dados PostgreSQL"""
    return psycopg2.connect(**DB_CONFIG)

def limpar_lista_string(valor):
    """Converte strings com listas para listas Python"""
    if pd.isna(valor) or valor == '' or valor == '...':
        return []
    
    # Converte para string se for número (corrige o erro do .strip())
    valor = str(valor).strip()
    
    # Se o valor é apenas um número ou valor simples, retorna lista vazia
    if valor.replace('.', '').replace('-', '').isdigit():
        return []
    
    # Se já é uma lista JSON válida
    if valor.startswith('[') and valor.endswith(']'):
        try:
            return json.loads(valor.replace("'", '"'))
        except:
            pass
    
    # Tenta extrair valores entre aspas
    matches = re.findall(r"'([^']*)'|\"([^\"]*)\"", valor)
    if matches:
        resultado = [m[0] or m[1] for m in matches if m[0] or m[1]]
        # Filtra valores vazios ou inválidos
        return [item for item in resultado if item and item.strip() and item.strip() != '...']
    
    # Fallback: split por vírgula
    if ',' in valor:
        items = [item.strip().strip("'\"[]") for item in valor.split(',') if item.strip()]
        return [item for item in items if item and item != '...']
    
    # Se não tem vírgula e não está entre aspas, verifica se é um valor válido
    valor_limpo = valor.strip("'\"[]")
    if valor_limpo and valor_limpo != '...' and not valor_limpo.replace('.', '').replace('-', '').isdigit():
        return [valor_limpo]
    
    return []

def converter_boolean(valor):
    """Converte valores para boolean"""
    if pd.isna(valor):
        return False
    return str(valor).lower() in ['true', '1', 'yes', 't']

def converter_numero(valor, tipo='int', padrao=0):
    """Converte valores para números, tratando erros"""
    if pd.isna(valor) or valor == '' or valor == '...':
        return padrao
    try:
        if tipo == 'int':
            return int(float(valor))
        elif tipo == 'float':
            return float(valor)
    except:
        return padrao

def inserir_entidade_relacionamento(cursor, tabela_entidade, tabela_relacao, 
                                    coluna_nome, valores, app_id, id_coluna='id'):
    """Insere entidades e seus relacionamentos com jogos"""
    if not valores:
        return
    
    ids_inseridos = []
    
    for valor in valores:
        # Converte para string e remove espaços
        valor = str(valor).strip()
        
        if not valor or valor == '' or valor == '...' or valor == 'nan':
            continue
        
        # Verifica se a entidade já existe
        cursor.execute(f"SELECT {id_coluna} FROM {tabela_entidade} WHERE {coluna_nome} = %s", (valor,))
        resultado = cursor.fetchone()
        
        if resultado:
            entidade_id = resultado[0]
        else:
            # Insere nova entidade
            if id_coluna == 'id' and tabela_entidade != 'developers':
                cursor.execute(
                    f"INSERT INTO {tabela_entidade} ({coluna_nome}) VALUES (%s) RETURNING {id_coluna}",
                    (valor,)
                )
                entidade_id = cursor.fetchone()[0]
            else:
                # Para developers, precisa gerar ID manualmente
                cursor.execute(f"SELECT COALESCE(MAX({id_coluna}), 0) + 1 FROM {tabela_entidade}")
                entidade_id = cursor.fetchone()[0]
                cursor.execute(
                    f"INSERT INTO {tabela_entidade} ({id_coluna}, {coluna_nome}) VALUES (%s, %s)",
                    (entidade_id, valor)
                )
                ids_inseridos.append(entidade_id)
                continue
        
        ids_inseridos.append(entidade_id)
    
    # Insere relacionamentos
    if ids_inseridos:
        # Determina os nomes das colunas na tabela de relacionamento
        if 'language' in tabela_relacao:
            col_id, col_app_id = 'id_language', 'app_id_game'
        elif 'developer' in tabela_relacao:
            col_id, col_app_id = 'id_developer', 'app_id_game'
        elif 'publisher' in tabela_relacao:
            col_id, col_app_id = 'id_publisher', 'app_id_game'
        elif 'categorie' in tabela_relacao:
            col_id, col_app_id = 'id_categorie', 'app_id_game'
        elif 'genre' in tabela_relacao:
            col_id, col_app_id = 'id_genre', 'app_id_game'
        elif 'tag' in tabela_relacao:
            col_id, col_app_id = 'id_tag', 'app_id_game'
        elif 'screenshot' in tabela_relacao:
            col_id, col_app_id = 'id_screenshot', 'app_id_game'
        elif 'movie' in tabela_relacao:
            col_id, col_app_id = 'id_movie', 'app_id_game'
        
        valores_relacao = [(id_ent, app_id) for id_ent in ids_inseridos]
        execute_values(
            cursor,
            f"INSERT INTO {tabela_relacao} ({col_id}, {col_app_id}) VALUES %s ON CONFLICT DO NOTHING",
            valores_relacao
        )

def processar_csv(caminho_csv):
    """Processa o CSV e insere os dados no banco"""
    # Lê o CSV com encoding adequado para caracteres especiais
    print("Tentando ler o arquivo CSV...")
    
    try:
        df = pd.read_csv(caminho_csv, encoding='latin-1')
        print("✓ Arquivo lido com sucesso usando encoding: latin-1")
    except Exception as e:
        print(f"Erro com latin-1, tentando cp1252... ({e})")
        try:
            df = pd.read_csv(caminho_csv, encoding='cp1252')
            print("✓ Arquivo lido com sucesso usando encoding: cp1252")
        except Exception as e:
            print(f"Erro com cp1252, tentando iso-8859-1... ({e})")
            df = pd.read_csv(caminho_csv, encoding='iso-8859-1')
            print("✓ Arquivo lido com sucesso usando encoding: iso-8859-1")
    
    print(f"Total de linhas no CSV: {len(df)}")
    
    conn = conectar_db()
    cursor = conn.cursor()
    
    try:
        linhas_processadas = 0
        linhas_com_erro = 0
        erros_detalhados = []
        
        for idx, row in df.iterrows():
            try:
                app_id = converter_numero(row['AppID'], 'int')
                
                if app_id == 0:
                    print(f"Linha {idx}: AppID inválido, pulando...")
                    linhas_com_erro += 1
                    continue
                
                # Verifica se o jogo já existe
                cursor.execute("SELECT app_id FROM games WHERE app_id = %s", (app_id,))
                if cursor.fetchone():
                    print(f"Linha {idx}: Jogo {app_id} já existe, pulando...")
                    continue
                
                # Insere na tabela games
                cursor.execute("""
                    INSERT INTO games (app_id, name, release_date, required_age, price, 
                                       discount, dlc_count, about_the_game, website, 
                                       header_image, suport_url, suport_email)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                    app_id,
                    str(row['Name']) if pd.notna(row['Name']) else 'Unknown',
                    str(row['Release date']) if pd.notna(row['Release date']) else '0',
                    converter_numero(row['Required age'], 'int', 0),
                    converter_numero(row['Price'], 'float', 0.0),
                    converter_numero(row.get('Discount', 0), 'int', 0),
                    converter_numero(row.get('DLC count', 0), 'int', 0),
                    str(row['About the game']) if pd.notna(row['About the game']) else None,
                    str(row.get('Website', '')) if pd.notna(row.get('Website')) else None,
                    str(row.get('Header image', '')) if pd.notna(row.get('Header image')) else None,
                    str(row.get('Support url', '')) if pd.notna(row.get('Support url')) else None,
                    str(row.get('Support email', '')) if pd.notna(row.get('Support email')) else None
                ))
                
                # Insere na tabela game_metrics
                cursor.execute("""
                    INSERT INTO game_metrics (app_id, estimated_owners, peak_ccu, reviews, 
                                              metacritic_score, metacritic_url, user_score, 
                                              positive, negative, score_rank, acheivements, 
                                              recomendations, avarege_palytime_forever, 
                                              avarege_playtime_two_weeks, media_playtime_forever, 
                                              median_playtime_two_weeks)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                    app_id,
                    str(row['Estimated owners']) if pd.notna(row['Estimated owners']) else '0',
                    converter_numero(row['Peak CCU'], 'int', 0),
                    str(row.get('Reviews', '')) if pd.notna(row.get('Reviews')) else None,
                    converter_numero(row.get('Metacritic score', 0), 'int', 0),
                    str(row.get('Metacritic url', '')) if pd.notna(row.get('Metacritic url')) else None,
                    converter_numero(row.get('User score', 0), 'int', 0),
                    converter_numero(row.get('Positive', 0), 'int', 0),
                    converter_numero(row.get('Negative', 0), 'int', 0),
                    converter_numero(row.get('Score rank', 0), 'float', None),
                    converter_numero(row.get('Achievements', 0), 'int', 0),
                    converter_numero(row.get('Recommendations', 0), 'int', 0),
                    converter_numero(row['Average playtime forever'], 'int', 0),
                    converter_numero(row['Average playtime two weeks'], 'int', 0),
                    converter_numero(row['Median playtime forever'], 'int', 0),
                    converter_numero(row['Median playtime two weeks'], 'int', 0)
                ))
                
                # Insere na tabela plataforms
                cursor.execute("""
                    INSERT INTO plataforms (app_id, windows, mac, linux)
                    VALUES (%s, %s, %s, %s)
                """, (
                    app_id,
                    converter_boolean(row.get('Windows', False)),
                    converter_boolean(row.get('Mac', False)),
                    converter_boolean(row.get('Linux', False))
                ))
                
                # Processa Languages
                languages = limpar_lista_string(row['Supported languages'])
                inserir_entidade_relacionamento(
                    cursor, 'languages', 'languages_games', 
                    'language', languages, app_id
                )
                
                # Processa Developers
                developers = limpar_lista_string(row['Developers'])
                inserir_entidade_relacionamento(
                    cursor, 'developers', 'developers_games', 
                    'developer', developers, app_id
                )
                
                # Processa Publishers
                publishers = limpar_lista_string(row['Publishers'])
                inserir_entidade_relacionamento(
                    cursor, 'publishers', 'publishers_games', 
                    'publisher', publishers, app_id
                )
                
                # Processa Categories
                categories = limpar_lista_string(row['Categories'])
                inserir_entidade_relacionamento(
                    cursor, 'categories', 'categories_games', 
                    'categorie', categories, app_id
                )
                
                # Processa Genres
                genres = limpar_lista_string(row['Genres'])
                inserir_entidade_relacionamento(
                    cursor, 'genres', 'genres_games', 
                    'genre', genres, app_id
                )
                
                # Processa Tags
                tags = limpar_lista_string(row['Tags'])
                inserir_entidade_relacionamento(
                    cursor, 'tags', 'tags_games', 
                    'tags', tags, app_id
                )
                
                # Processa Screenshots
                screenshots = limpar_lista_string(row.get('Screenshots', ''))
                inserir_entidade_relacionamento(
                    cursor, 'screenshots', 'screenshots_games', 
                    'screenshot', screenshots, app_id
                )
                
                # Processa Movies
                movies = limpar_lista_string(row.get('Movies', ''))
                inserir_entidade_relacionamento(
                    cursor, 'movies', 'movies_games', 
                    'movie', movies, app_id
                )
                
                linhas_processadas += 1
                
                if linhas_processadas % 100 == 0:
                    conn.commit()
                    print(f"Processadas {linhas_processadas} linhas...")
                    
            except Exception as e:
                erro_msg = f"Linha {idx} (AppID: {row.get('AppID', 'N/A')}): {str(e)}"
                print(erro_msg)
                erros_detalhados.append(erro_msg)
                linhas_com_erro += 1
                conn.rollback()
                continue
        
        conn.commit()
        print(f"\n=== Resumo ===")
        print(f"Linhas processadas com sucesso: {linhas_processadas}")
        print(f"Linhas com erro: {linhas_com_erro}")
        
        if erros_detalhados:
            print(f"\n=== Detalhes dos Erros ===")
            for erro in erros_detalhados:
                print(erro)
        
    except Exception as e:
        print(f"Erro crítico: {str(e)}")
        conn.rollback()
    finally:
        cursor.close()
        conn.close()

if __name__ == "__main__":
    # Substitua pelo caminho do seu arquivo CSV
    caminho_csv = "Caminho para o seu csv"
    
    print("Iniciando processamento do CSV...")
    processar_csv(caminho_csv)
    print("Processamento concluído!")